In [1]:
import tensorflow as tf
import os
import requests
import io
import PIL
import PIL.Image 
import math
import pathlib
from tensorflow.keras import layers
import numpy as np


In [2]:
batch_size = 8 # 每⼀批所处理的图⽚数量
img_height = 160 # 图⽚⾼度，单位为像素
img_width = 160 # 图⽚宽度，单位为像素


TF_SERVING_BASE_URL = 'http://localhost:8501/'
image_dir = './datasets/handworks'

task_id = '350002'
task_dir = os.path.join('./datasets/handworks', task_id)
train_dir = os.path.join(task_dir, 'train')
test_dir = os.path.join(task_dir, 'test')

In [3]:
import shutil

id_code_mapping = { # 根据web界⾯显示情况设定
'0' : '0',
'1' : '1', 
}
total_image_count = len(list(pathlib.Path(image_dir).glob('*/*.png')))
print(total_image_count)
print(train_dir)

for img in pathlib.Path(image_dir).glob('*/*.png'): 
    if np.random.ranf() < 0.8: # 需要通过修改数值来控制测试集的⽐例
      dst_dir = os.path.join(train_dir, os.path.basename(os.path.dirname(img)))
    else:
      dst_dir = os.path.join(test_dir, os.path.basename(os.path.dirname(img)))
    if not os.path.exists(dst_dir):
      os.makedirs(dst_dir)
    #print(os.path.dirname(img))
    dst = os.path.join(dst_dir, img.name)
    #print(dst)
    #dst = os.path.join(dst_dir, record['id']+'.png')
    #img.save(dst)
    shutil.copy(img, dst)

85
./datasets/handworks/350002/train


In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
validation_split=0.2, # 设定验证集⽐例
subset="training",
seed=123,
#image_size=(img_height, img_width),
batch_size=batch_size)

Found 85 files belonging to 2 classes.
Using 68 files for training.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
validation_split=0.2, # 设定验证集⽐例
subset="validation",
seed=123,
#image_size=(img_height, img_width),
batch_size=batch_size)

Found 85 files belonging to 2 classes.
Using 17 files for validation.


In [6]:
print(train_ds.class_names)
#print(train_ds.shape)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  #print(image_batch[0])  
  print(labels_batch[7])  
  break

['0', '1']
(8, 256, 256, 3)
(8,)
tf.Tensor(1, shape=(), dtype=int32)


In [7]:
import matplotlib.pyplot as plt

#plt.figure(figsize=(20, 10))
#for images, labels in train_ds.take(1):
    #for i in range(20):
        #ax = plt.subplot(5, 10, i + 1)
        #plt.imshow(images[i].numpy().astype("uint8"))
        #plt.title(train_ds.class_names[labels[i]])
        #plt.axis("off")
        #print(labels[i])

In [8]:

num_classes = 2
model = tf.keras.Sequential([ # 根据需要调整模型结构
layers.experimental.preprocessing.Resizing(img_height, img_width),
tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    #tf.keras.layers.Conv2D(32, 3, activation='relu'),
    #tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
   
])


In [9]:
model.compile(
optimizer='adam',
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])



In [10]:
model.fit(
train_ds,
validation_data=val_ds,
epochs=20
)

print(model.evaluate(val_ds))

print(model.summary())


Epoch 1/20
9/9 [==============================] - 2s 104ms/step - loss: 3.4090 - accuracy: 0.5735 - val_loss: 0.6311 - val_accuracy: 0.5882
Epoch 2/20
9/9 [==============================] - 1s 92ms/step - loss: 0.4968 - accuracy: 0.7941 - val_loss: 0.4805 - val_accuracy: 0.8824
Epoch 3/20
9/9 [==============================] - 1s 93ms/step - loss: 0.3602 - accuracy: 0.8235 - val_loss: 0.6314 - val_accuracy: 0.7059
Epoch 4/20
9/9 [==============================] - 1s 91ms/step - loss: 0.3886 - accuracy: 0.8235 - val_loss: 0.3381 - val_accuracy: 0.8824
Epoch 5/20
9/9 [==============================] - 1s 88ms/step - loss: 0.2058 - accuracy: 0.9412 - val_loss: 0.1658 - val_accuracy: 1.0000
Epoch 6/20
9/9 [==============================] - 1s 94ms/step - loss: 0.0766 - accuracy: 1.0000 - val_loss: 0.0834 - val_accuracy: 1.0000
Epoch 7/20
9/9 [==============================] - 1s 86ms/step - loss: 0.0288 - accuracy: 1.0000 - val_loss: 0.0371 - val_accuracy: 1.0000
Epoch 8/20
9/9 [==========

In [11]:
tf.keras.models.save_model(
model,
'./models/image/4/', # ./models为tensorflow-serving的模型根⽬录
overwrite=True,
include_optimizer=True,
save_format=None,
signatures=None,
options=None
)

INFO:tensorflow:Assets written to: ./models/image/4/assets


In [12]:
import numpy as np
import json
def test_image_model(test_dir,  code, batch_size=10): 
  imgs = []
  codes = []
  index  = 0
  imgdir = os.path.join(test_dir, val_ds.class_names[code]) 
  for i in pathlib.Path(imgdir).glob('./*.png'): 
    #print(i.name)
    index = index + 1
    imgs = []
    img = PIL.Image.open(i)
    pixels = np.array(img)
    #plt.imshow(pixels)
    if (index ==2):
      img.show()
    img2 = img.rotate(45)
    img2 = tf.image.resize(pixels,[200,200])
    img2 = tf.image.random_crop(img2,[160,160,3])
    img2 =PIL.Image.fromarray(np.uint8(img2))
    if (index==2):
      img2.show()
    imgs.append(pixels.tolist()) 
    img.close()
    for j in range(int(math.ceil(len(imgs)/batch_size))):
      req_data = json.dumps({
        'inputs': imgs[j*batch_size:(j+1)*batch_size],
      })               
    response = requests.post(TF_SERVING_BASE_URL+'v1/models/image/versions/4:predict', # 根据部署地址填写
    data=req_data,
    headers={"content-type": "application/json"})
    #print(response.status_code)
    if response.status_code != 200:
      raise RuntimeError('Request tf-serving failed: ' + response.text)
    resp_data = json.loads(response.text) 
    if 'outputs' not in resp_data \
         or type(resp_data['outputs']) is not list:
      raise ValueError('Malformed tf-serving response')
    codes.extend(np.argmax(resp_data['outputs'], axis=1).tolist())
    #print("class:"+ str(np.argmax(resp_data['outputs'])))
  return codes



In [13]:
codes = test_image_model(test_dir,  0)
print('类别0的准确率', 1 - round(np.sum(codes)/len(codes),4))
codes = test_image_model(test_dir, 1)
print('类别1的准确率', round(np.sum(codes)/len(codes),4))

ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/image/versions/4:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa18550fd60>: Failed to establish a new connection: [Errno 61] Connection refused'))